In [1]:
import mitsuba as mi
import drjit as dr
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random 
import imageio
import cv2
import numpy as np

In [2]:
scene_path = os.path.join('', 'scenes') 
results_spatio_path = os.path.join('','results_spatio') 
mi.set_variant('scalar_rgb')
texture_path = os.path.join(scene_path,'textures')

tokki = os.path.join(scene_path,'meshes/bunny.ply')


In [3]:
cam1 = mi.load_dict({
'type': 'perspective',
'fov': 45, #24.6 mm
'to_world': mi.ScalarTransform4f.look_at(
    origin=[0.03, 0, -0.7],  # increase the z-coordinate to move the camera farther away
    target=[0.03, 1, -15],
    up=[0, 0, 1]
),
'film2': {
    'type': 'hdrfilm',
    'width': 360,
    'height': 360,
}
# 'distortion': [0, 0, 0, 0, 0]
})

In [12]:
import cgi, os
files = os.listdir('dir/graycode_pattern/')

for f in files:
    s = "dir/graycode_pattern/"
    s = s + f
    scene1 = mi.load_dict({
        'type':'scene',
        'integrator': {'type': 'direct'},
        'light': {
            'type': 'projector',
            'irradiance': {
                'type': 'bitmap',
                'filename': s, 
            },
            'fov': 45,
            'to_world': mi.ScalarTransform4f.look_at(
                origin=[-0.03, 0.1, -0.7],
                target=[-0.03, -0.1, -15],
                up=[0, 0, 1]
            )
            },
        'tok':{
            'type': 'ply',
            'filename': tokki,
            'to_world': mi.ScalarTransform4f.translate([0, 0, -1]),
            'bsdf': {
                'type': 'diffuse',
                'reflectance': {'type': 'rgb', 'value': [0.1, 0.2, 0.3]},
            }
        }
    })

    image = mi.render(scene=scene1, sensor=cam1, spp=64)
    image = np.clip(image * 1000, 0, 1000).astype(np.uint8)
    st = "./results/resultfar_" + f
    # Save the image as a PNG file using imageio
    imageio.imwrite(st, image)


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 4.978283882141113]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 4.611562728881836]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 4.978283882141113]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 4.978283882141113]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 4.621075630187988]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 4.94192361831665]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 4.977836

In [5]:
import numpy as np
from math import tan, radians

# Define the field of view in degrees
FOV = 45.0

# Define the focal length in mm

f_x = 50
f_y = 50
c_x = 180
c_y = 180

cam_int = np.array([[f_x, 0, c_x],
              [0, f_y, c_y],
              [0, 0, 1]])
cam_int

array([[ 50,   0, 180],
       [  0,  50, 180],
       [  0,   0,   1]])

In [6]:
focal_length = 50
image_width = 360
image_height = 360

f_x = focal_length
f_y = focal_length
x_0 = image_width / 2
y_0 = image_height / 2
s = 0

proj_int = np.array([[f_x, s, x_0],
                     [0, f_y, y_0],
                     [0, 0, 1]])
proj_int


array([[ 50.,   0., 180.],
       [  0.,  50., 180.],
       [  0.,   0.,   1.]])

In [7]:
import numpy as np

# Define the camera and projector positions and orientations
cam_pos = np.array([0.03, 0, -0.7])
cam_target = np.array([0.03, 1, -15])
cam_up = np.array([0, 0, 1])

proj_pos = np.array([-0.03, 0.1, -0.7])
proj_target = np.array([-0.03, -0.1, -15])
proj_up = np.array([0, 0, 1])

# Compute the camera and projector axes
cam_dir = cam_target - cam_pos
cam_dir /= np.linalg.norm(cam_dir)

cam_right = np.cross(cam_dir, cam_up)
cam_right /= np.linalg.norm(cam_right)

cam_up = np.cross(cam_right, cam_dir)

proj_dir = proj_target - proj_pos
proj_dir /= np.linalg.norm(proj_dir)

proj_right = np.cross(proj_dir, proj_up)
proj_right /= np.linalg.norm(proj_right)

proj_up = np.cross(proj_right, proj_dir)

# Build the camera and projector rotation matrices
cam_proj_rmat = np.zeros((3, 3))
cam_proj_rmat[:, 0] = cam_right
cam_proj_rmat[:, 1] = cam_up
cam_proj_rmat[:, 2] = cam_dir

proj_cam_rmat = np.zeros((3, 3))
proj_cam_rmat[:, 0] = proj_right
proj_cam_rmat[:, 1] = proj_up
proj_cam_rmat[:, 2] = proj_dir

# Build the camera and projector translation vectors
cam_trans = -cam_proj_rmat @ cam_pos
proj_trans = -proj_cam_rmat @ proj_pos

# Build the camera and projector extrinsic matrices
cam_ext_mat = np.eye(4)
cam_ext_mat[:3, :3] = cam_proj_rmat
cam_ext_mat[:3, 3] = cam_trans

proj_ext_mat = np.eye(4)
proj_ext_mat[:3, :3] = proj_cam_rmat
proj_ext_mat[:3, 3] = proj_trans

cam_proj_rmat = np.linalg.inv(cam_ext_mat[:3, :3]) @ proj_ext_mat[:3, :3]

cam_proj_rmat

array([[-1.        ,  0.        ,  0.        ],
       [ 0.        , -0.99649071, -0.08370346],
       [ 0.        , -0.08370346,  0.99649071]])

In [8]:
# Get the rotation matrix from the camera transformation matrix
# import mitsuba as mi
# cam_transform = mi.ScalarTransform4f.look_at(
#     origin=[0.03, 0, -0.7],
#     target=[0.03, 1, -15],
#     up=[0, 0, 1]
# )
# cam_int = np.array([[322.6663776454747, 0, 180],
#        [0, 322.6663776454747, 180],
#        [0, 0.00e+00, 1]])
# proj_int = cam_int
# cam_to_world = np.array(cam_transform.matrix)
# cam_rot_vec, _ = cv2.Rodrigues(cam_to_world[:3, :3])
# cam_rot_mat = np.array(cam_rot_vec)

# # Define the proj_dist and proj_int matrices
# proj_dist = np.zeros((4, 1))

# proj =  mi.ScalarTransform4f.look_at(
#             origin=[-0.03, 0.1, -0.7],
#             target=[-0.03, -0.1, -15],
#             up=[0, 0, 1]
#         )

proj_int = np.array([[ 50.,   0., 180.],
       [  0.,  50., 180.],
       [  0.,   0.,   1.]])
# Compute the cam_proj_rmat matrix
# cam_proj_rmat = np.dot(proj_int, np.hstack((cam_rot_mat, -np.dot(cam_rot_mat, cam_to_world[:3, 3].reshape(-1,1)))))


cam_proj_rmat = np.dot(proj_int, cam_int)

cam_proj_rmat

##50mm
##snesor sensor_size = (2 * 50mm * tan(45/2 degrees))





array([[2.50e+03, 0.00e+00, 9.18e+03],
       [0.00e+00, 2.50e+03, 9.18e+03],
       [0.00e+00, 0.00e+00, 1.00e+00]])

In [9]:
u = 360
v = 360
point = np.array([u, v, 1])
cam_int = np.array([[322.6663776454747, 0, 180],
       [0, 322.6663776454747, 180],
       [0, 0.00e+00, 1]])
proj_int = cam_int
cam_proj_tvec = -np.linalg.inv(cam_proj_rmat) @ np.linalg.inv(cam_int) @ proj_int @ point
cam_proj_tvec

array([ 3.528,  3.528, -1.   ])

In [10]:
cam_int = np.array([[322.6663776454747, 0, 180],
       [0, 322.6663776454747, 180],
       [0, 0.00e+00, 1]])


In [11]:
print(params['PerspectiveCamera.to_world'])


NameError: name 'params' is not defined